In [ ]:
!pip install -q langchain==0.1.16 langchain-openai==0.0.8 langchain-community==0.0.32

In [ ]:
!pip install --upgrade numpy -q

In [3]:
from langchain_openai import ChatOpenAI
import os
import pandas as pd

from langchain.agents import AgentType, initialize_agent,load_tools
from langchain.tools import Tool


In [ ]:
# # local model
# llm = ChatOpenAI(
#     openai_api_base="http://localhost:1234/v1",
#     openai_api_key="lm_studio",
#     model="llama-3.2-1b-instruct",
#     temperature=0.9
# )

In [7]:
llm = ChatOpenAI(
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key="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",
    model="google/gemma-3-27b-it:free",
    temperature=0.5
)

os.environ["SERPAPI_API_KEY"] = "a1a578596b6a0d1371670a504c20074c5d2c07320a8af3b91fdc5239ee1b7e3a"


In [ ]:
!git clone https://github.com/deepanrajm/deep_learning.git

In [5]:
EXCEL_PATH = "/content/deep_learning/agents/expense.xlsx"

In [6]:
def analyze_expenses_excel(query: str) -> str:
    """
    Analyze salary and expenses from Excel using pandas.
    """
    df = pd.read_excel(EXCEL_PATH)

    salary = df.loc[df["Category"].str.lower() == "salary", "Monthly_Amount"].sum()

    expenses_df = df[df["Category"].str.lower() != "salary"]

    total_expenses = expenses_df["Monthly_Amount"].sum()
    savings = salary - total_expenses
    savings_percentage = (savings / salary) * 100 if salary > 0 else 0

    top_expenses = (
        expenses_df
        .sort_values(by="Monthly_Amount", ascending=False)
        .head(3)[["Category", "Monthly_Amount"]]
        .to_dict(orient="records")
    )

    summary = {
        "monthly_salary": salary,
        "total_monthly_expenses": total_expenses,
        "monthly_savings": savings,
        "savings_percentage": round(savings_percentage, 2),
        "top_3_expense_categories": top_expenses
    }

    return str(summary)


In [8]:
expense_tool = Tool(
    name="expense_analyzer",
    description=(
        "Analyze salary and monthly expenses from Excel and return "
        "savings, spending breakdown, and high expense categories."
    ),
    func=analyze_expenses_excel
)


In [ ]:
pip install google-search-results

In [10]:
tools = load_tools(
    ["serpapi", "llm-math"],
    llm=llm
)

tools.append(expense_tool)


In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)


In [ ]:
query = """
This is my monthly salary and expense sheet.
Please analyze it and tell me:
1. How much I am saving
2. Where I am overspending
3. How I can reduce expenses
4. How to improve my savings
"""

result = agent.run(query)
print(result)
